In [1]:
import pandas as pd
import numpy as np
from py2neo import Graph
from py2neo import Node
from py2neo import Relationship

# Commuting flow preprocessing
<BR>
<A HREF="https://www.census.gov/data/tables/2015/demo/metro-micro/commuting-flows-2015.html">Commuting Flows Data</A>

The data is read. The missing values result from lines indicating commuting flows to other workplaces outside the USA. These rows are deleted.

In [2]:
commuting_flow=pd.read_excel("RawData/commuting_flows.xlsx",dtype={'State FIPS Code': object,'County FIPS Code':object,
                                                                   'State FIPS Code.1':object,
                                                                   'County FIPS Code.1':object})
commuting_flow.drop(columns=[" Margin of Error"], inplace=True)
commuting_flow.dropna(inplace=True)

Combining state and county FIPS code to one FIPS code

In [3]:
commuting_flow['State FIPS Code.1']=commuting_flow['State FIPS Code.1'].str[1:]
commuting_flow["State FIPS Code"]=commuting_flow["State FIPS Code"]+commuting_flow["County FIPS Code"]
commuting_flow["State FIPS Code.1"]=commuting_flow["State FIPS Code.1"]+commuting_flow["County FIPS Code.1"]
commuting_flow.drop(columns=['County FIPS Code.1','County FIPS Code'],inplace=True)
commuting_flow.rename(columns={'State Name':'ResState','County Name':'ResCounty','State Name.1':'PoWState',
                              'County Name.1':'PoWCounty','Workers in Commuting Flow':'Workers',
                              'State FIPS Code.1':'PoWGEOID','State FIPS Code':'ResGEOID'},inplace=True)
commuting_flow['ResCounty']=commuting_flow['ResCounty'].str.replace(' County', '')
commuting_flow['PoWCounty']=commuting_flow['PoWCounty'].str.replace(' County', '')

In [4]:
commuting_flow.to_csv('Data_input_Neo4j/WorkPlaceFlows_clean.csv',index=False)

# Preparing graph based on commuting flows between states

We only consider the commuting flow between states.

In [5]:
between_state=commuting_flow[commuting_flow["ResState"]!=commuting_flow["PoWState"]]

We compute the nodes and edges and save the results in the folder Data_input_neo4j

In [6]:
commuting_edges=between_state.groupby(["ResState","PoWState"],as_index=False).agg({'Workers':'sum'})
commuting_nodes=pd.DataFrame(commuting_edges['ResState'].unique(),columns=['State'])

In [7]:
commuting_edges.to_csv('Data_input_neo4j/commuting_edges.csv',index=False)
commuting_nodes.to_csv('Data_input_neo4j/commuting_nodes.csv',index=False)

## Commuting flow degree centrality

<div class="alert alert-block alert-info"> Make sure that all datafiles that are in the folder Data_input_neo4j are copied into the input folder of your database.</div>

## Make graph

In [8]:
graph = Graph("bolt://localhost:7687", password='xxx', name='neo4j')

Insert the nodes and relationships.

In [9]:
graph.run("MATCH (n) DETACH DELETE n")
query="""LOAD CSV WITH HEADERS FROM $file as row
        with row
        CALL apoc.create.node(['State','Place'],{name:row.State}) YIELD node
        RETURN distinct true"""
graph.run(query,file='file:///commuting_nodes.csv')
query="""LOAD CSV WITH HEADERS FROM $file as row
        with row
        MATCH (source:State{name:row.ResState})
        MATCH (target:State{name:row.PoWState})
        CALL apoc.create.relationship(source,"COMMUTING",{weight:toFloat(row.Workers)},target) YIELD rel
        RETURN distinct true"""
graph.run(query,file='file:///commuting_edges.csv')

true
true


Create an in-memory graph, called 'Commuting'.

In [10]:
query="""CALL gds.graph.project('Commuting','State',{COMMUTING:{properties:'weight', orientation:'UNDIRECTED'}})"""
graph.run(query)

nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{State: {label: 'State', properties: {}}}","{COMMUTING: {orientation: 'UNDIRECTED', aggregation: 'DEFAULT', type: 'COMMUTING', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}}",Commuting,52,4748,22


## Perform degree centrality

We can use the gds.degree.stream algorithm to calculate the degree centrality of the states.

In [11]:
query="""CALL gds.degree.stream('Commuting',{relationshipWeightProperty:'weight'})
        YIELD nodeId, score
        RETURN gds.util.asNode(nodeId).name AS Place, score AS Degree_Centrality_com
        ORDER by Degree_Centrality_com desc"""
result = graph.run(query).data() 
df_degree_commuting=pd.DataFrame(result)

The in-memory graph is dropped again.

In [12]:
graph.run("CALL gds.graph.drop('Commuting')")

graphName,database,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema
Commuting,neo4j,,-1,52,4748,"{relationshipProjection: {COMMUTING: {orientation: 'UNDIRECTED', aggregation: 'DEFAULT', type: 'COMMUTING', properties: {weight: {defaultValue: null, property: 'weight', aggregation: 'DEFAULT'}}}}, nodeProjection: {State: {label: 'State', properties: {}}}, relationshipProperties: [], creationTime: datetime('2022-06-06T08:42:17.940273600+02:00'), validateRelationships: false, readConcurrency: 4, sudo: false, nodeProperties: [], username: null}",1.790346907993967,datetime('2022-06-06T08:42:17.940273600+02:00'),datetime('2022-06-06T08:42:17.960229800+02:00'),"{relationships: {COMMUTING: {weight: 'Float (DefaultValue(NaN), PERSISTENT, Aggregation.DEFAULT)'}}, nodes: {State: {}}}"


The results are saved into a csv file.

In [13]:
df_degree_commuting.to_csv('PreprocessedData/degree_commuting_between_states.csv',index=False)

# Preparing graph based on commuting flows between counties


<div class="alert alert-block alert-info"> Make sure that all datafiles that are in the folder Data_input_neo4j are copied into the input folder of your database.</div>

In [14]:
graph.run("MATCH(n) DETACH DELETE n")

(No data)

In [15]:
query="""
LOAD CSV WITH HEADERS FROM "file:///WorkplaceFlows_clean.csv" as row
MERGE (:County {geoid: row.ResGEOID, county: row.ResCounty, state: row.ResState})
"""
graph.run(query)
print("Nodes inserted")

Nodes inserted


In [16]:
query="""
CREATE INDEX INDX IF NOT EXISTS
FOR (n:County)
ON (n.geoid)
"""
graph.run(query)
print("Index created")

Index created


In [17]:
query="""
LOAD CSV WITH HEADERS FROM "file:///WorkplaceFlows_clean.csv" as row
MERGE (res:County {geoid: row.ResGEOID, county: row.ResCounty, state: row.ResState})
MERGE (pow:County {geoid: row.PoWGEOID, county: row.PoWCounty, state: row.PoWState})
CREATE (res)-[:COMMUTES {workers: toFloat(row.Workers), res: row.ResGEOID, pow: row.PoWGEOID}]->(pow)
"""
graph.run(query)
print("Relationships created")

Relationships created


In [18]:
# Quick test to make sure things are connected and working, should be two Albany counties, one in NY, one in WY
query = """
MATCH (a:County) WHERE a.county = 'Albany' RETURN a.geoid, a.state;
"""
graph.run(query)

a.geoid,a.state
36001,New York
56001,Wyoming


In [19]:
query = """
CALL gds.graph.project('counties','County',{COMMUTES:{properties:'workers'}});
"""
graph.run(query)

nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{County: {label: 'County', properties: {}}}","{COMMUTES: {orientation: 'NATURAL', aggregation: 'DEFAULT', type: 'COMMUTES', properties: {workers: {defaultValue: null, property: 'workers', aggregation: 'DEFAULT'}}}}",counties,3220,137806,73


## Centrality Measures
<A HREF="https://neo4j-website.s3.eu-central-1.amazonaws.com/build/html/Algorithms/centrality/centrality.html">Centrality Measures</A>

In [20]:
# Counties with highest centrality score
query = """
CALL gds.beta.closeness.stream('counties')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).geoid AS geoid, 
       gds.util.asNode(nodeId).county AS county,
       gds.util.asNode(nodeId).state AS state,
       score AS centralScore
ORDER BY score DESC;
"""
result=graph.run(query)
centrality = result.to_data_frame()
centrality.head(10)

,geoid,county,state,centralScore
0,48201,Harris,Texas,0.574054
1,17031,Cook,Illinois,0.556574
2,48113,Dallas,Texas,0.536524
3,13121,Fulton,Georgia,0.534830
4,11001,District of Columbia,District of Columbia,0.532263
5,06037,Los Angeles,California,0.532087
6,27053,Hennepin,Minnesota,0.527031
7,47157,Shelby,Tennessee,0.522665
8,04013,Maricopa,Arizona,0.521056
9,37119,Mecklenburg,North Carolina,0.520466


In [21]:
#centrality.to_csv("PreprocessedData/centrality.csv", index=False)

In [22]:
# Counties with highest betweeness score
query = """
CALL gds.betweenness.stream('counties')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).geoid AS geoid, 
       gds.util.asNode(nodeId).county AS county,
       gds.util.asNode(nodeId).state AS state,
       score AS betweenScore
ORDER BY score DESC;
"""
result=graph.run(query)
betweeness = result.to_data_frame()
betweeness.head(10)

,geoid,county,state,betweenScore
0,48201,Harris,Texas,617958.281608
1,17031,Cook,Illinois,469159.093054
2,48113,Dallas,Texas,301811.660055
3,06037,Los Angeles,California,270725.273712
4,04013,Maricopa,Arizona,255235.306000
5,13121,Fulton,Georgia,245228.297783
6,27053,Hennepin,Minnesota,243818.914986
7,48439,Tarrant,Texas,223837.763622
8,47157,Shelby,Tennessee,203355.954000
9,37119,Mecklenburg,North Carolina,192922.170706


In [23]:
#betweeness.to_csv("PreprocessedData/betweeness.csv", index=False)

In [24]:
# Counties with highest page rank score
query = """
CALL gds.pageRank.stream('counties',{maxIterations: 50,  dampingFactor: 0.85,relationshipWeightProperty:'workers'})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).geoid AS geoid, 
       gds.util.asNode(nodeId).county AS county,
       gds.util.asNode(nodeId).state AS state,
       score AS pageRank
ORDER BY score DESC;
"""
result=graph.run(query)
pagerank = result.to_data_frame()
pagerank.head(10)

,geoid,county,state,pageRank
0,72127,San Juan Municipio,Puerto Rico,16.812835
1,48201,Harris,Texas,16.499235
2,13121,Fulton,Georgia,14.583588
3,11001,District of Columbia,District of Columbia,13.112132
4,36061,New York,New York,12.787671
5,27053,Hennepin,Minnesota,12.269300
6,47157,Shelby,Tennessee,11.608050
7,17031,Cook,Illinois,11.504621
8,48113,Dallas,Texas,11.472922
9,21111,Jefferson,Kentucky,11.419643


In [25]:
#pagerank.to_csv("PreprocessedData/pagerank.csv", index=False)

## Community Detection
<A HREF="https://neo4j-website.s3.eu-central-1.amazonaws.com/build/html/Algorithms/community/community.html">Class Notes on Community Detection</A>

<A HREF="https://neo4j.com/docs/graph-data-science/current/algorithms/louvain/">Cypher Louvain Modularity</A><BR>

In [26]:
# Louvain Modularity 
# Louvain quantifies how well a node is assigned to a group by looking at the density of connections within 
# a cluster in comparison to an average or random sample
query = """
CALL gds.louvain.stream('counties',{
  includeIntermediateCommunities: true,
  relationshipWeightProperty:'workers'
})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).geoid AS geoid, 
       gds.util.asNode(nodeId).county AS county,
       gds.util.asNode(nodeId).state AS state,
       communityId, 
       intermediateCommunityIds
ORDER BY communityId ASC;
"""
result=graph.run(query)
louvainMod = result.to_data_frame()
louvainMod.head(10)

,geoid,county,state,communityId,intermediateCommunityIds
0,19041,Clay,Iowa,20,[20]
1,19063,Emmet,Iowa,20,[20]
2,19059,Dickinson,Iowa,20,[20]
3,19147,Palo Alto,Iowa,20,[20]
4,20137,Norton,Kansas,24,[24]
5,20147,Phillips,Kansas,24,[24]
6,20183,Smith,Kansas,24,[24]
7,31043,Dakota,Nebraska,28,[28]
8,19133,Monona,Iowa,28,[28]
9,19149,Plymouth,Iowa,28,[28]


In [27]:
#louvainMod.to_csv("PreprocessedData/louvainmod.csv", index=False)

<A HREF="https://neo4j.com/docs/graph-data-science/current/algorithms/label-propagation/">Cypher Label Propagation</A><BR>

In [28]:
# Label Propagation
# In LPA, nodes select their group based on their direct neighbors. 
# This process is well suited to networks where groupings are less clear and weights can 
# be used to help a node determine which community to place itself within
query = """
CALL gds.labelPropagation.stream('counties',{
maxIterations: 10,
relationshipWeightProperty:'workers'
})
YIELD nodeId, communityId
RETURN collect(gds.util.asNode(nodeId).geoid) AS geoid,
       collect(gds.util.asNode(nodeId).county) AS county,
       collect(gds.util.asNode(nodeId).state) AS state,
       communityId AS label
ORDER BY size(county) DESC;
"""
result=graph.run(query)
labelprop = result.to_data_frame()
labelprop.head(10)

,geoid,county,state,label
0,"[72137, 72051, 72033, 72119, 72143, 72045, 720...","[Toa Baja Municipio, Dorado Municipio, Cataño ...","[Puerto Rico, Puerto Rico, Puerto Rico, Puerto...",8857
1,"[28033, 47167, 47157, 47047, 28009, 28093]","[DeSoto, Tipton, Shelby, Fayette, Benton, Mars...","[Mississippi, Tennessee, Tennessee, Tennessee,...",6739
2,"[22033, 22063, 22091, 22037, 22121]","[East Baton Rouge Parish, Livingston Parish, S...","[Louisiana, Louisiana, Louisiana, Louisiana, L...",6537
3,"[13079, 13021, 13207, 13169, 13289]","[Crawford, Bibb, Monroe, Jones, Twiggs]","[Georgia, Georgia, Georgia, Georgia, Georgia]",6716
4,"[18109, 18063, 18097, 18081, 18059]","[Morgan, Hendricks, Marion, Johnson, Hancock]","[Indiana, Indiana, Indiana, Indiana, Indiana]",6818
5,"[39049, 39041, 39045, 39117, 39129]","[Franklin, Delaware, Fairfield, Morrow, Pickaway]","[Ohio, Ohio, Ohio, Ohio, Ohio]",6663
6,"[47065, 13083, 13295, 47153, 13047]","[Hamilton, Dade, Walker, Sequatchie, Catoosa]","[Tennessee, Georgia, Georgia, Tennessee, Georgia]",6706
7,"[40143, 40131, 40145, 40037, 40113]","[Tulsa, Rogers, Wagoner, Creek, Osage]","[Oklahoma, Oklahoma, Oklahoma, Oklahoma, Oklah...",6725
8,"[21111, 21103, 21215, 21185, 21029]","[Jefferson, Henry, Spencer, Oldham, Bullitt]","[Kentucky, Kentucky, Kentucky, Kentucky, Kentu...",6758
9,"[13101, 13185, 13027, 13173]","[Echols, Lowndes, Brooks, Lanier]","[Georgia, Georgia, Georgia, Georgia]",6802


In [29]:
#labelprop.to_csv("PreprocessedData/labelprop.csv", index=False)

<A HREF="https://neo4j.com/docs/graph-data-science/current/algorithms/wcc/">Cypher Weakly Connected Components</A><BR>

In [30]:
# Weakly Connected Components (SCC)
query = """
CALL gds.wcc.stream('counties',{relationshipWeightProperty:'workers'})
YIELD nodeId, componentId
RETURN gds.util.asNode(nodeId).geoid AS geoid, 
       gds.util.asNode(nodeId).county AS county,
       gds.util.asNode(nodeId).state AS state,
       componentId AS Component
ORDER BY Component, county;
"""
result=graph.run(query)
wcc = result.to_data_frame()
wcc.head(10)
# WCC return all counties in one component, thus not very interesting results

,geoid,county,state,Component
0,45001,Abbeville,South Carolina,0
1,22001,Acadia Parish,Louisiana,0
2,51001,Accomack,Virginia,0
3,16001,Ada,Idaho,0
4,21001,Adair,Kentucky,0
5,19001,Adair,Iowa,0
6,29001,Adair,Missouri,0
7,40001,Adair,Oklahoma,0
8,38001,Adams,North Dakota,0
9,39001,Adams,Ohio,0


In [31]:
query = """
CALL gds.wcc.stats('counties',{relationshipWeightProperty:'workers'})
YIELD componentCount
"""
graph.run(query)
# yup, only 1 component

componentCount
1


<A HREF="https://neo4j.com/docs/graph-data-science/current/algorithms/modularity-optimization/">Cypher Modularity Optimization</A> aka Girvan-Newman

In [32]:
query = """
CALL gds.beta.modularityOptimization.stream('counties',{relationshipWeightProperty:'workers'})
YIELD nodeId, communityId
RETURN nodeId,
       gds.util.asNode(nodeId).geoid AS geoid, 
       gds.util.asNode(nodeId).county AS county,
       gds.util.asNode(nodeId).state AS state, 
       communityId
ORDER BY nodeId
"""
graph.run(query)

nodeId,geoid,county,state,communityId
0,19003,Adams,Iowa,1619
1,19009,Audubon,Iowa,1253
2,19071,Fremont,Iowa,1253


<A HREF="https://neo4j.com/docs/graph-data-science/current/algorithms/triangle-count/">Cypher Triangle Counts</A><BR>
This may not work for directed graphs, but here's a bunch of code anyway.

In [35]:
# General Triangle Count stats
query="""CALL gds.graph.project('counties_undirected','County',{COMMUTES:{orientation:'UNDIRECTED'}})"""
graph.run(query)
query = """
CALL gds.triangleCount.stats('counties_undirected')
YIELD globalTriangleCount,nodeCount
"""
graph.run(query)


globalTriangleCount,nodeCount
1061847,3220


In [36]:
# Print Triangles
query = """
CALL gds.triangleCount.stream('counties_undirected')
YIELD nodeId , triangleCount
RETURN gds.util.asNode(nodeId).geoid as geoid,
       gds.util.asNode(nodeId).county as county,
       triangleCount as nbr_triangles
ORDER BY triangleCount DESC;
"""
graph.run(query)

geoid,county,nbr_triangles
17031,Cook,34071
48201,Harris,31656
48113,Dallas,22538


In [37]:
query = """
CALL gds.alpha.triangles('counties_undirected')
YIELD nodeA, nodeB, nodeC
RETURN gds.util.asNode(nodeA).county AS nodeA,
       gds.util.asNode(nodeB).county AS nodeB,
       gds.util.asNode(nodeC).county AS nodeC;
"""
graph.run(query)

nodeA,nodeB,nodeC
Adams,Audubon,Story
Fremont,Page,Atchison
Fremont,Page,Nodaway


<A HREF="https://neo4j.com/docs/graph-data-science/current/algorithms/local-clustering-coefficient/">Cypher Local Clustering Coeff</A>

In [38]:
# Clustering Coefficient - does not support Directed 
query = """
CALL gds.localClusteringCoefficient.stream('counties_undirected')
YIELD nodeId, localClusteringCoefficient
WHERE localClusteringCoefficient > 0
RETURN gds.util.asNode(nodeId).geoid as geoid,
       gds.util.asNode(nodeId).county as county,
       gds.util.asNode(nodeId).state as state,
       localClusteringCoefficient
ORDER BY localClusteringCoefficient DESC;
"""
graph.run(query)

geoid,county,state,localClusteringCoefficient
06091,Sierra,California,0.8214285714285714
72037,Ceiba Municipio,Puerto Rico,0.7867647058823529
06011,Colusa,California,0.7806267806267806


In [39]:
query = """
CALL gds.graph.drop('counties');
"""
graph.run(query)
query = """
CALL gds.graph.drop('counties_undirected');
"""
graph.run(query)

graphName,database,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema
counties_undirected,neo4j,,-1,3220,275612,"{relationshipProjection: {COMMUTES: {orientation: 'UNDIRECTED', aggregation: 'DEFAULT', type: 'COMMUTES', properties: {}}}, nodeProjection: {County: {label: 'County', properties: {}}}, relationshipProperties: [], creationTime: datetime('2022-06-06T08:43:00.916208100+02:00'), validateRelationships: false, readConcurrency: 4, sudo: false, nodeProperties: [], username: null}",0.02659017981356812,datetime('2022-06-06T08:43:00.916208100+02:00'),datetime('2022-06-06T08:43:00.945162300+02:00'),"{relationships: {COMMUTES: {}}, nodes: {County: {}}}"
